In [1]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import models
import torchvision.transforms as T
import torchvision.datasets as dset
import numpy as np
import random

# import torch.nn.functional as F
# import torchvision

In [2]:
# Hyper Parameter 超參數

NUM_EPOCHS = 20
BATCH_SIZE = 16  # DataLoader每次抓取的數量
LEARNING_RATE = 1e-4

# 其他可調參數
PRINT_EVERY = 100


In [3]:
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [4]:
FOLDERPATH = 'Colab\ Notebooks/Poster/a.after_test'

In [5]:
%cd drive/MyDrive/$FOLDERPATH

/content/drive/.shortcut-targets-by-id/1OC3Immm4L7H1Rp4Po9pQSFc3-pDwQpi4/Poster/a.after_test


In [6]:
%ls

'Create train & test dataset.ipynb'   EfficientNet_b4_9244.ipynb   test/    weights/
 EfficientNet_b0_929.ipynb            Ensemble_0929.ipynb          train/
 EfficientNet_b4_0929.ipynb           ResNet50_0929.ipynb          val/


In [7]:
#Check if gpu is available
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f'計算裝置:{device}')

計算裝置:cuda


In [8]:
# 添加DataLoader的random seed
# 方法來源：https://yanwei-liu.medium.com/pytorch-reproducibility-db8458111b75
# Ensure that the output of all random values ​​is consistent

def set_seed(seed=42, loader=None):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  try:
    loader.sampler.generator.manual_seed(seed)
  except AttributeError:
    pass

set_seed()

In [9]:
# Data pre-processing
transform_turing = T.Compose([
    T.Resize((300,300)),
    T.RandomHorizontalFlip(p=0.5),  # 水平翻轉
    T.RandomAutocontrast(),  # 自動調整圖像的對比度
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 幫助模型學習到不同顏色和光照條件下的圖像特徵 # 隨機調整亮度、對比度、飽和度、色調
    T.RandomRotation(10),  # 隨機將圖像在[-10度, 10度]範圍內進行旋轉
    T.RandomCrop((224,224)),
    T.ToTensor()
])
transform_new = T.Compose([
    T.Resize((300,300)),
    T.RandomCrop((224,224)),
    T.ToTensor()
])
transform_test = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

In [10]:
# Load train data
# 0: portrait  1: SD
train_data_1 = dset.ImageFolder('../Dataset/2_classes/train', transform=transform_turing)
train_data_2 = dset.ImageFolder('../new_test', transform=transform_new)
train_data_3 = dset.ImageFolder('../new_test', transform=transform_turing)
train_data_4 = dset.ImageFolder('train', transform=transform_new)
train_data_5 = dset.ImageFolder('train', transform=transform_turing)
train_data = ConcatDataset([train_data_1, train_data_2, train_data_3, train_data_4, train_data_5])

In [11]:
# Load val and test Data
# 0: portrait  1: SD
val_data = dset.ImageFolder('../Dataset/2_classes/val', transform=transform_test)
test_data = dset.ImageFolder('../Dataset/2_classes/test', transform=transform_test)

In [12]:
# Check data dimension
print(train_data[0][0].shape)
print(val_data[0][1])

torch.Size([3, 224, 224])
0


In [13]:
NUM_TRAIN = len(train_data)
NUM_VAL = len(val_data)
NUM_TEST = len(test_data)
print('Number of training:', NUM_TRAIN)
print('Number of validation:', NUM_VAL)
print('Numver of test:', NUM_TEST)

Number of training: 1856
Number of validation: 80
Numver of test: 80


In [14]:
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [15]:
# Create Mini-Batch
from torch.utils.data import DataLoader
mini_trains = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
mini_vals = DataLoader(val_data, batch_size=BATCH_SIZE, worker_init_fn=seed_worker, generator=g)
mini_test = DataLoader(test_data, batch_size=BATCH_SIZE)

print(mini_trains)

In [16]:
# # Check data in mini_trains
# iterable = iter(mini_trains)
# x, y = next(iterable)

# iterable = iter(mini_vals)
# x, y = next(iterable)

In [17]:
# # # input data
# # x.shape
# # y.shape

# # Check data dimension
# print(train_data[0][0].shape)
# print(val_data[0][1])

In [18]:
# Building model EfficientNet
efficientnet = models.efficientnet_b4(pretrained = True)
# print(efficientnet)
num_flatten = efficientnet.classifier[1].in_features # 獲取 EfficientNet 最後一層的輸入特徵數量
efficientnet.classifier[1] = nn.Linear(num_flatten, 2) # 替換為適應2元分類的新線性層（2個輸出）
model = efficientnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b4_rwightman-23ab8bcd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b4_rwightman-23ab8bcd.pth
100%|██████████| 74.5M/74.5M [00:00<00:00, 186MB/s]


In [19]:
# Define loss function & optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4 )

In [20]:
# Training procedure
def train(model, mini_trains, optimizer, device, NUM_EPOCHS):
  for epoch in range(NUM_EPOCHS):
    for count, (x, y) in enumerate(mini_trains):
        model.train() #Turn on training mode
        # Move data to device
        x = x.to(device)
        y = y.to(device)
        # ForwardProp
        scores = model(x)
        # Calculate loss
        loss = loss_function(scores, y)
        if count % PRINT_EVERY == 0:
            print(f'[Epoch {epoch+1}] Training Loss:', loss.item())
            validation(mini_vals, epoch, model, device)
        # Clear the previous gradients
        optimizer.zero_grad()
        # Get the gradients
        loss.backward()
        # Update the weights
        optimizer.step()
    print('-'*50)

In [21]:
# Validation Procedure
def validation(mini_vals, epoch, model, device):
  # Turn on val mode
  model.eval()
  with torch.no_grad():
    Val_Acc = 0
    for x, y in mini_vals:
      # move data to device
      x = x.to(device)
      y = y.to(device)
      # Forward Prop
      scores = model(x)
      # Calculate accuracy
      predictions = scores.max(1)[1] # scores.argmax(1)
      acc = predictions.eq(y).sum().item()
      Val_Acc += acc
    print(f'[Epoch {epoch+1}] Accuracy:', Val_Acc/NUM_VAL)

In [22]:
train(model, mini_trains, optimizer, device, NUM_EPOCHS)

[Epoch 1] Training Loss: 0.6819508075714111
[Epoch 1] Accuracy: 0.4875
[Epoch 1] Training Loss: 0.6817237734794617
[Epoch 1] Accuracy: 0.725
--------------------------------------------------
[Epoch 2] Training Loss: 0.6620318293571472
[Epoch 2] Accuracy: 0.7375
[Epoch 2] Training Loss: 0.4993453025817871
[Epoch 2] Accuracy: 0.7375
--------------------------------------------------
[Epoch 3] Training Loss: 0.42024821043014526
[Epoch 3] Accuracy: 0.7625
[Epoch 3] Training Loss: 0.37569132447242737
[Epoch 3] Accuracy: 0.8
--------------------------------------------------
[Epoch 4] Training Loss: 0.08042982220649719
[Epoch 4] Accuracy: 0.825
[Epoch 4] Training Loss: 0.24605460464954376
[Epoch 4] Accuracy: 0.875
--------------------------------------------------
[Epoch 5] Training Loss: 0.08960310369729996
[Epoch 5] Accuracy: 0.9
[Epoch 5] Training Loss: 0.07550579309463501
[Epoch 5] Accuracy: 0.8625
--------------------------------------------------
[Epoch 6] Training Loss: 0.03440422564

In [23]:
# Test score procedure
def test(mini_tests, model, device):
  # Use val mode to test
  model.eval()
  with torch.no_grad():
    Val_Acc = 0
    for x, y in mini_tests:
      # move data to device
      x = x.to(device)
      y = y.to(device)
      # Last Forward Prop
      score_test = model(x)
      # Calculate Accuracy
      predictions = score_test.max(1)[1]
      acc = predictions.eq(y).sum().item()
      Val_Acc += acc
    print(f'[Final] Test Accuracy:', Val_Acc/NUM_TEST)

In [24]:
test(mini_test, model, device)

[Final] Test Accuracy: 0.9125


In [25]:
# save trained weights
torch.save(efficientnet.state_dict(), 'weights/EfficientNetb4_weights_mirrorup_0930.pth')

In [26]:
# # Load model and trained weights
# efficientnet = models.efficientnet_b4(pretrained=False)  #改 False

# num_flatten = efficientnet.classifier[1].in_features # 獲取 EfficientNet 最後一層的輸入特徵數量
# efficientnet.classifier[1] = nn.Linear(num_flatten, 2) # 替換為適應2元分類的新線性層（2個輸出）

# PATH = 'EfficientNetb4_weights_920.pth'
# efficientnet.load_state_dict(torch.load(PATH))

# model = efficientnet.to(device)

In [27]:
# Load test data
new_test_data = dset.ImageFolder('../milestone', transform=transform_test)
NUM_NEW_TEST = len(new_test_data)
print('Number of new test:', NUM_NEW_TEST)
mini_tests = DataLoader(new_test_data, batch_size=1, worker_init_fn=seed_worker, generator=g)

Number of new test: 400


In [28]:
new_test_data[0][0].shape

torch.Size([3, 224, 224])

In [29]:
# Test score procedure
def milestone_test(mini_tests, model, device):
    # Use eval mode to test
    model.eval()
    #valid_classes = [0, 2]  # Classes Filter
    with torch.no_grad():
        acc_count = 0
        total_samples = 0
        for x, y in mini_tests:
            # move data to device
            x = x.to(device)
            y = y.to(device)

            # Last Forward Prop
            score_test = model(x)

            # Calculate predictions
            predictions = score_test.max(1)[1]

            # Calculate Accuracy
            acc = predictions.eq(y).sum().item()
            acc_count += acc

        print(f'[Final] Test Accuracy:', acc_count / NUM_NEW_TEST)

In [30]:
milestone_test(mini_tests, model, device)

[Final] Test Accuracy: 0.85
